In [22]:
import polars as pl
from src.tank_ts_endpoint import fetch_tank_ts_data, GetTanksTimeDataReq

req = GetTanksTimeDataReq(scada_id='9881904')

df = await fetch_tank_ts_data(req)
if df is None:
    raise ValueError("df is None")
lf = df.lazy()
lf.collect()

ts,value,scada_id,tank_name,uom
datetime[μs],f64,str,str,str
2021-07-15 11:24:32.025842,76.742959,"""9881904""","""OilTank4Level""","""in"""
2021-07-15 10:24:32.025842,76.636488,"""9881904""","""OilTank4Level""","""in"""
2021-07-15 09:24:32.025842,76.569618,"""9881904""","""OilTank4Level""","""in"""
2021-07-15 08:24:32.025842,76.5744,"""9881904""","""OilTank4Level""","""in"""
2021-07-15 07:24:32.025842,76.675313,"""9881904""","""OilTank4Level""","""in"""
2021-07-15 06:24:32.025842,76.889969,"""9881904""","""OilTank4Level""","""in"""
2021-07-15 05:24:32.025842,77.229802,"""9881904""","""OilTank4Level""","""in"""
2021-07-15 04:24:32.025842,77.700715,"""9881904""","""OilTank4Level""","""in"""
2021-07-15 03:24:32.025842,78.303709,"""9881904""","""OilTank4Level""","""in"""


In [21]:
pl.Config.set_tbl_rows(1500)

polars.config.Config

In [17]:
lf = lf.drop("key_metric")
lf.collect()

ts,value,scada_id,tank_name,uom
datetime[μs],f64,str,str,str
2021-07-15 11:24:32.025842,76.742959,"""9881904""","""OilTank4Level""","""in"""
2021-07-15 10:24:32.025842,76.636488,"""9881904""","""OilTank4Level""","""in"""
2021-07-15 09:24:32.025842,76.569618,"""9881904""","""OilTank4Level""","""in"""
2021-07-15 08:24:32.025842,76.5744,"""9881904""","""OilTank4Level""","""in"""
2021-07-15 07:24:32.025842,76.675313,"""9881904""","""OilTank4Level""","""in"""
…,…,…,…,…
2021-01-30 17:12:37.864857,308.522869,"""9881904""","""OilTank4Volume""","""bbl"""
2021-01-30 16:12:37.864857,306.875488,"""9881904""","""OilTank4Volume""","""bbl"""
2021-01-30 15:12:37.864857,305.19665,"""9881904""","""OilTank4Volume""","""bbl"""


In [18]:
tank_metrics = ["Level", "Volume", "InchesUntilAlarm", "InchesToESD"]
tank_types = ["Water", "Oil"]

tank_metrics_str= "|".join(tank_metrics)
tank_types_str = "|".join(tank_types)

pattern = f'^(?<is_esd>ESD-)?(?<tank_type>{tank_types_str})Tank(?<tank_number>[0-9]*)(?<tank_metric>{tank_metrics_str})'
lf = lf.with_columns(separated_metrics = pl.col("tank_name").str.extract_groups(pattern))
lf = lf.unnest("separated_metrics")
lf.collect()

ts,value,scada_id,tank_name,uom,is_esd,tank_type,tank_number,tank_metric
datetime[μs],f64,str,str,str,str,str,str,str
2021-07-15 11:24:32.025842,76.742959,"""9881904""","""OilTank4Level""","""in""",null,"""Oil""","""4""","""Level"""
2021-07-15 10:24:32.025842,76.636488,"""9881904""","""OilTank4Level""","""in""",null,"""Oil""","""4""","""Level"""
2021-07-15 09:24:32.025842,76.569618,"""9881904""","""OilTank4Level""","""in""",null,"""Oil""","""4""","""Level"""
2021-07-15 08:24:32.025842,76.5744,"""9881904""","""OilTank4Level""","""in""",null,"""Oil""","""4""","""Level"""
2021-07-15 07:24:32.025842,76.675313,"""9881904""","""OilTank4Level""","""in""",null,"""Oil""","""4""","""Level"""
…,…,…,…,…,…,…,…,…
2021-01-30 17:12:37.864857,308.522869,"""9881904""","""OilTank4Volume""","""bbl""",null,"""Oil""","""4""","""Volume"""
2021-01-30 16:12:37.864857,306.875488,"""9881904""","""OilTank4Volume""","""bbl""",null,"""Oil""","""4""","""Volume"""
2021-01-30 15:12:37.864857,305.19665,"""9881904""","""OilTank4Volume""","""bbl""",null,"""Oil""","""4""","""Volume"""


In [19]:
lf = lf.drop("tank_name", "is_esd", "tank_type", "tank_number")
lf.collect()

ts,value,scada_id,uom,tank_metric
datetime[μs],f64,str,str,str
2021-07-15 11:24:32.025842,76.742959,"""9881904""","""in""","""Level"""
2021-07-15 10:24:32.025842,76.636488,"""9881904""","""in""","""Level"""
2021-07-15 09:24:32.025842,76.569618,"""9881904""","""in""","""Level"""
2021-07-15 08:24:32.025842,76.5744,"""9881904""","""in""","""Level"""
2021-07-15 07:24:32.025842,76.675313,"""9881904""","""in""","""Level"""
…,…,…,…,…
2021-01-30 17:12:37.864857,308.522869,"""9881904""","""bbl""","""Volume"""
2021-01-30 16:12:37.864857,306.875488,"""9881904""","""bbl""","""Volume"""
2021-01-30 15:12:37.864857,305.19665,"""9881904""","""bbl""","""Volume"""


In [20]:
aggregated_lf = lf.group_by(["tank_metric", "uom"]).agg([
    pl.col("ts").alias("timestamps"),
    pl.col("value").alias("values")
])

aggregated_lf.collect()

tank_metric,uom,timestamps,values
str,str,list[datetime[μs]],list[f64]
"""Volume""","""bbl""","[2021-02-01 12:12:37.864857, 2021-02-01 11:12:37.864857, … 2021-01-30 13:12:37.864857]","[300.070112, 299.96364, … 301.785926]"
"""Level""","""in""","[2021-07-15 11:24:32.025842, 2021-07-15 10:24:32.025842, … 2021-07-13 12:24:32.025842]","[76.742959, 76.636488, … 78.458773]"
